In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import walk
import pickle
import os

In [2]:
batch_files = []
data_files  = []

for (dirpath, dirnames, fnames) in walk('dat'):
    for f in fnames:
        if 'Batch_' in f:
            batch_files.append(f)
        else:
            data_files.append(f)

In [3]:
batch_files, data_files

(['Batch_4318249_batch_results.csv',
  'Batch_4319068_batch_results.csv',
  'Batch_4319396_batch_results.csv',
  'Batch_4322859_batch_results.csv',
  'Batch_4323327_batch_results.csv',
  'Batch_4324179_batch_results.csv'],
 ['test (1).xlsx', 'test.xlsx'])

In [4]:
df_batch = []
df_dat = []
df_comment = []

for f in batch_files:
    df_batch.append(pd.read_csv('dat/' + f))

for f in data_files:
    with pd.ExcelFile('dat/' + f, engine='openpyxl') as xls:
        df_dat.append(pd.read_excel(xls, 'DB'))
        df_comment.append(pd.read_excel(xls, 'COMMENTS'))

# concatenate all data files in a single dataframe
df_dat_whole = pd.DataFrame(columns=df_dat[0].columns)

for dat in df_dat:
    df_dat_whole = df_dat_whole.append(dat)

In [5]:
df_comment[0].columns

Index(['id', 'user', 'comment'], dtype='object')

In [6]:
# compile a user comment dataframe
new_df_comment = pd.DataFrame(columns=df_comment[0].columns)

for df in df_comment:
    new_df_comment = new_df_comment.append(df, ignore_index=True)

In [7]:
df_batch[0].columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Answer.surveycode',
       'Approve', 'Reject'],
      dtype='object')

In [8]:
# worker batch file
new_df_batch = pd.DataFrame(columns=['WorkerId', 'WorkTimeInSeconds' , 'Answer.surveycode'])

for i in df_batch:
    temp = i[['WorkerId', 'WorkTimeInSeconds' , 'Answer.surveycode']]
    new_df_batch = new_df_batch.append(temp, ignore_index=True)

old_df_batch = new_df_batch.copy()

new_df_batch

,WorkerId,WorkTimeInSeconds,Answer.surveycode
0,A65N7P7LMZ4GE,4587,8701315a-59b9-4fa6-9f5f-4c85ad143a76
1,A2VRDE2FHCBMF8,1474,a2cdf1da-5aa1-40c8-94ed-471a4b1fd78d
2,A34CPKFZXBX1PO,1255,eb0162e1-dd64-4d42-a44a-f7154ae858bc
3,AG9LMLEPXP2YC,4801,36c04674-62c8-4f4c-a1b1-ec6a12badf01
4,AFSW5HVZ0EKET,3046,94d0f9fd-14dd-4191-963e-e4b145625acc
...,...,...,...
145,A19ZWBQT8A3LIR,2222,6264fbc4-f71d-4fbf-ad47-8593724dc742
146,A38DXFI1TZA295,5894,520a33dc-5e7f-44f3-85e7-968a80d5ba52
147,A25R2OI9L2Q1OW,5615,7aae45c9-1aa0-415b-a60a-70deda6437f2
148,A3I40B0FATY8VH,2722,189efc73-db8a-428d-bcf8-c20ce9ab883a


In [9]:
#find any duplicates
new_df_batch[new_df_batch['WorkerId'].duplicated(keep=False)]

,WorkerId,WorkTimeInSeconds,Answer.surveycode
12,A1PTH9KTRO06EG,2233,2f05d213-d605-4d99-9e96-7e6fbffe8114
31,A1PTH9KTRO06EG,6742,7a5bd8df-e71c-4c5f-bcb4-75e23efe3a8c


In [10]:
# drop duplicate workers
new_df_batch = new_df_batch.drop_duplicates('WorkerId')

In [11]:
codes = new_df_batch['Answer.surveycode']
new_df_dat = pd.DataFrame(columns=df_dat[0].columns)

for code in codes:
    if len(code) != 36: #screen out wrong survey code submissions       
        print('Submission Error: {}'.format(code))
        # print('{}: {}'.format(set(new_df_batch[new_df_batch['user'] == code]['user'])), code)
    else:
        for dat in df_dat:
            temp = dat.loc[dat['user'] == code]
            if temp.size != 0:
                temp = temp.drop_duplicates('trial') #removing duplicates likely due to server errors (brings n>100 to n=100)
                new_df_dat = new_df_dat.append(temp)
                pass    

Submission Error: AKSJ3C5O3V9RB (worker ID submitted due to technical difficulties, will message code, please do not reject, thank you very much.)


In [12]:
print('There are {} total MTurkers\nThere are {} Worker IDs\nThere are {} matching completion codes\nThere are {} total registration'.format(old_df_batch.shape[0], new_df_batch.shape[0], new_df_dat['user'].unique().size, df_dat_whole['user'].unique().size))

There are 150 total MTurkers
There are 149 Worker IDs
There are 148 matching completion codes
There are 168 total registration


In [13]:
df_dat_whole = df_dat[0].append(df_dat[1])

newlist = set(df_dat_whole['user'])
for lst in newlist:
    print('{}: {}'.format(lst, df_dat_whole[df_dat_whole['user'] == lst]['trial'].size))

9de97d9d-dead-4f5b-9ac4-30953edc8521: 100
1b3ecbf2-324d-43d5-a0f1-636d2e8077eb: 100
7dce86f8-81d4-410e-a69b-07dceae49984: 100
12756fb7-1181-409e-97f3-fe99ea8e5313: 100
84b846d3-6a28-4c3b-814f-eaca107e61fd: 100
0ec33481-2993-41dd-b4ea-356ae1640b82: 8
ce002466-ba12-475f-b8fe-2fdc675164aa: 100
d96b10b3-b36b-4616-ac35-b408559a737b: 1
422de2ad-6830-4fb6-b855-2d614e455bb1: 100
e2b7745a-81ff-48b8-8334-1708180c58d4: 100
c57d0f35-0130-42f8-851a-ff5f0a70a86c: 100
3c4ad156-8243-41ce-9586-898d3ed51eb8: 100
7600cbb5-e672-430c-96f0-dbaee82ce76c: 100
f554e046-a38f-4914-a6d1-65d70f3901a1: 100
d351e46b-9a67-4fa5-b45f-3110fd01ef4e: 100
d77604e1-cb54-47ac-a425-a1fe2e652624: 100
20732ea8-cba0-4509-b94f-7b8814fbc098: 100
8701315a-59b9-4fa6-9f5f-4c85ad143a76: 100
55ba7440-718e-425e-aee9-8ceb5842b193: 100
c413b1c4-b834-4e2b-afb5-4a7b2c29a818: 100
60a1d705-82cb-4919-ac39-f2fd4fa861e9: 100
9bbc2220-1321-4f13-bab5-b762e310b7c1: 101
66c1de63-6325-44fc-8a5e-cbbc562d1bc9: 100
50a28b14-4f4e-42e4-9b0e-a0931c6067d1: 

In [14]:
# output final raw database as an excel file (this does not screen out participants who failed to pass the catch trial)
new_df_dat.to_excel('MTurk_data.xlsx', index=False)

In [15]:
# making a score profile
lst_id = set(new_df_dat['user'])

df_score = pd.DataFrame(columns=['user', 'score', 'sample', 'comment'])

for userid in lst_id:
    temp = new_df_dat.loc[new_df_dat['user'] == userid]['score']
    max_score = temp.max()
    sample = temp.size
    if new_df_comment.loc[new_df_comment['user'] == userid]['comment'].size != 0:
        temp = new_df_comment.loc[new_df_comment['user'] == userid]['comment'].iloc[0]
        if str(temp) != 'nan':
            comment = temp
        else:
            comment = 'N/A'
    else:
        comment = 'N/A'
    df_score = df_score.append(pd.DataFrame([[userid, max_score, sample, comment]], columns=['user', 'score', 'sample', 'comment']))

In [16]:
df_score.to_excel('MTurk_score_profile.xlsx', index=False)

In [17]:
df_score.loc[df_score['sample'] != 100]

,user,score,sample,comment
0,36c04674-62c8-4f4c-a1b1-ec6a12badf01,4,99,N/A
0,d1da6636-c4d9-4391-b802-564fe2734f17,4,99,N/A


In [18]:
# disqualified list
df_score.loc[df_score['score'] < 4]

,user,score,sample,comment
0,ce002466-ba12-475f-b8fe-2fdc675164aa,0,100,good
0,f554e046-a38f-4914-a6d1-65d70f3901a1,1,100,"This was interesting, I have no idea how I did..."
0,d77604e1-cb54-47ac-a425-a1fe2e652624,3,100,"This was neat and it all worked well, thank you."
0,20732ea8-cba0-4509-b94f-7b8814fbc098,2,100,Interesting.
0,c413b1c4-b834-4e2b-afb5-4a7b2c29a818,2,100,N/A
0,0965127f-3b1c-4e0b-b89c-4253940b9bda,3,100,I hope I did a decent job on your HIT. Thank y...
0,1ea752f4-be23-4aa6-a039-7e014600b4d7,2,100,good work
0,c1605a02-6872-4ea9-ae2f-bf015d97ac3c,3,100,It was pretty hard to know what the patterns w...
0,e35a6adb-7dfb-455b-8940-b8bda9e847d1,0,100,good
0,b889a15a-1b6e-4270-808c-23fa9fd33582,3,100,good task


In [19]:
#exporting user comments only
pd.set_option("max_colwidth", 100)

valid_comments = pd.DataFrame(df_score[df_score['comment'] != 'N/A']['comment']).reset_index(drop=True)
valid_comments.to_excel('User_comments.xlsx')
valid_comments.head(50)

,comment
0,I had to do it twice. The application broke about 25% into it the first time.
1,"I don't know if this had any impact on my responses, but the URL title stands out, ""inducedbias...."
2,good
3,No negative Comments. Everything will be alright.
4,I do not have any comments.
5,"This was interesting, I have no idea how I did. I would be interested in that"
6,That was hard!
7,"This was neat and it all worked well, thank you."
8,Interesting.
9,This was extremely slow loading. I ended up waiting for way too long between each trial.


In [20]:
bool_screen = (df_score['score'] >= 4) #* (df_score['sample'] == 100) #catch trial
id_list = df_score.loc[bool_screen]['user'].tolist() #make a ID list of the subjects meeting the criteria
print('There are {} subjects who passed the catch trials'.format(len(id_list)))

There are 126 subjects who passed the catch trials


In [21]:
new_df_dat_real = pd.DataFrame(columns=new_df_dat.columns)

for id_i in id_list:
    new_df_dat_real = new_df_dat_real.append(new_df_dat.loc[new_df_dat['user'] == id_i])

In [22]:
# compute euclidean distance
new_df_dat_real['d'] = '' # adding new column
new_df_dat_real['y'] = '' # separate column for coordinate

for i in range(new_df_dat_real.shape[0]):
    temp_char = new_df_dat_real.iloc[i]['x'][1:-1]
    comma = temp_char.rfind(',')
    x = float(temp_char[:comma])
    y = float(temp_char[comma+1:])
    new_df_dat_real['x'].iloc[i] = x
    new_df_dat_real['y'].iloc[i] = y
    new_df_dat_real['d'].iloc[i] = np.sqrt(x**2 + y**2)

In [23]:
#extra screening
# for user_i in new_df_dat_real[new_df_dat_real['mtype'] == 2][-300:-200]['user'].unique():
#     new_df_dat_real = new_df_dat_real[new_df_dat_real['user'] != user_i]

In [24]:
#extract [est, real, mtype, d]
output = np.array(new_df_dat_real[['est', 'real', 'mtype', 'x', 'd', 'y']])
output.shape

(12598, 6)

In [25]:
# final output of the cleaned database
filename = 'MTurk_ds.pickle'

with open(filename, 'wb') as f:
    pickle.dump(output, f)

In [26]:
#average time stayed on MTurk
new_df_batch.drop_duplicates('WorkerId')['WorkTimeInSeconds'].mean()/60

55.07997762863535

In [27]:
#average time took to complete
list_duration = []

for id_i in id_list:
    temp_df = new_df_dat_real[new_df_dat_real['user'] == id_i]
    t_1 = temp_df[temp_df['trial'] == 1]['date_created'].iloc[0]
    # t_50 = temp_df[temp_df['trial'] == 50]['date_created'].iloc[0]
    t_100 = temp_df[temp_df['trial'] == 100]['date_created'].iloc[0]
    list_duration.append((t_100-t_1).seconds)

np.mean(list_duration) / 60

16.304100529100527

## Update user list with do not repeat feature

In [28]:
filename = 'User_1237902_workers.csv'

df_MT = pd.read_csv(filename)
df_MT = df_MT.astype(object) #must be object otherwise MTurk does not recognize

In [29]:
df_new_MT = df_MT.copy()

workerid = new_df_batch['WorkerId'].tolist()
c = 0

for i in range(df_new_MT.shape[0]):
    
    if df_new_MT['Worker ID'][i] in workerid:
        df_new_MT['UPDATE-Already Participated JOVO'][i] = str(1)
        c += 1
        print('{}: {}'.format(c, df_new_MT['Worker ID'][i]))

1: A10JXOU89D5RXR
2: A12FCLCVIM2CL3
3: A12FTSX85NQ8N9
4: A13WTEQ06V3B6D
5: A153HLAVH5FILS
6: A17K1CHOI773VZ
7: A183WYXN12P2TJ
8: A18G2CLYSTENK
9: A1969Q0R4Y0E3J
10: A19ZWBQT8A3LIR
11: A1AZ97YYFVWFGC
12: A1CPY1HLCFTIL1
13: A1D6V3CKUT95LJ
14: A1DD23J1WBGQUU
15: A1DZMZTXWOM9MR
16: A1F5E941TP0UIZ
17: A1F9KLZGHE9DTA
18: A1GW09ZJF0RXTB
19: A1LA6CIGBNDOH9
20: A1LRJ4U04532TM
21: A1NLJ1L4VCQYV2
22: A1ODA3Q5H6HTDQ
23: A1P2RQ166VS5BT
24: A1P5FJPAEO3RP7
25: A1P6OXEJ86HQRM
26: A1PTH9KTRO06EG
27: A1QKIA8XRNEXIG
28: A1RV2LERVS0A4H
29: A1SH6BOQT9K5VT
30: A1SHLWKA0UH1IS
31: A1SLJKNSNHOJRN
32: A1W7I6FN183I8F
33: A1XVEKS9O73ERE
34: A1YZ0ETOCJO1B2
35: A1ZI4VU6650Q8B
36: A2196WCNDZULFS
37: A22DF0EWILGRLV
38: A22DVMN2Y3XHWA
39: A22T69YW4WUANF
40: A22W4Y4UI182L2
41: A256FHXGSY0E5D
42: A25R2OI9L2Q1OW
43: A26M997VYVK0E6
44: A26NGLGGFTATVN
45: A270HX8LH9LJ8W
46: A2BBJ85QRHJCQO
47: A2CEGS0NVDBCPT
48: A2CWADQFILGY80
49: A2EJ7U3TZAKROG
50: A2FYFCD16Z3PCC
51: A2G8FL7APWI0YS
52: A2GNG7DWHQA6ZZ
53: A2HU2D8889A2FV
54:

In [30]:
df_new_MT.to_csv(filename[:-4] + '_updated.csv', index=False)

In [31]:
# df_new_MT[df_new_MT['Worker ID'] == 'AY5WYV3L02AXW']
df_MT_updated = pd.DataFrame(columns=df_MT.columns)

for ids in new_df_batch['WorkerId']:
    df_MT_updated = df_MT_updated.append(df_new_MT[df_new_MT['Worker ID'] == ids])

df_MT_updated.head()

,Worker ID,Link to Individual Worker Page,Number of HITs approved or rejected - Lifetime,Number of HITs approved - Lifetime,Your Lifetime approval rate,Number of HITs approved or rejected - Last 30 days,Number of HITs approved - Last 30 days,Your Last 30 days approval rate,Number of HITs approved or rejected - Last 7 days,Number of HITs approved - Last 7 days,...,UPDATE-Completed Social Touch Study,CURRENT-Compensation HIT,UPDATE-Compensation HIT,CURRENT-Already Participated JOVO,UPDATE-Already Participated JOVO,CURRENT-Action Participation,UPDATE-Action Participation,CURRENT BlockStatus,UPDATE BlockStatus,BlockReason
1284,A65N7P7LMZ4GE,https://requester.mturk.com/workers/A65N7P7LMZ4GE,1,1,100.00%,1,1,100.00%,1,1,...,NaN,NaN,NaN,1.0,1,NaN,NaN,Never Blocked,NaN,NaN
812,A2VRDE2FHCBMF8,https://requester.mturk.com/workers/A2VRDE2FHCBMF8,2,2,100.00%,1,1,100.00%,1,1,...,NaN,NaN,NaN,1.0,1,NaN,NaN,Never Blocked,NaN,NaN
923,A34CPKFZXBX1PO,https://requester.mturk.com/workers/A34CPKFZXBX1PO,2,2,100.00%,1,1,100.00%,0,0,...,NaN,NaN,NaN,1.0,1,NaN,NaN,Never Blocked,NaN,NaN
1400,AG9LMLEPXP2YC,https://requester.mturk.com/workers/AG9LMLEPXP2YC,1,1,100.00%,1,1,100.00%,1,1,...,NaN,NaN,NaN,1.0,1,NaN,NaN,Never Blocked,NaN,NaN
1396,AFSW5HVZ0EKET,https://requester.mturk.com/workers/AFSW5HVZ0EKET,1,1,100.00%,1,1,100.00%,1,1,...,NaN,NaN,NaN,1.0,1,NaN,NaN,Never Blocked,NaN,NaN


In [32]:
# final check to make sure there is no duplicate
df_MT_updated['Worker ID'].unique().size

149